In [1]:
#!pip install simple_salesforce
#!pip install pymysql
#!pip install sqlalchemy

In [2]:
import json
from sqlalchemy import create_engine
from datetime import date
import pandas as pd
import pymysql
pymysql.install_as_MySQLdb()

In [3]:
# Make sure to use your own `config.py` file. Consider ensuring that these variable names are in sync
from config import sf_username, sf_password, sf_security_token
from config import remote_db_endpoint, remote_db_port
from config import remote_db_name, remote_db_user, remote_db_pwd

In [4]:
from simple_salesforce import Salesforce
sf = Salesforce(username=sf_username, password=sf_password, security_token=sf_security_token)

In [5]:
engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}")
conn = engine.connect()

## Prepare ETL for the Course data

In [6]:
course_data_df = pd.read_sql("SELECT * FROM course", conn)
course_data_df.head(30)

,ID_Course,CourseCode,CourseName,CreditHours,BootCampCourse
0,1,BC-DATAVIZ,Data Visualization and Analytics,12,1
1,2,BC-WEBDEV,Full Stack Web Development,12,1
2,3,BC-UIUX,User Interface/User Experience,12,1
3,4,CIS-349,Introduction to Databases,5,0
4,5,CIS-405,Database Programming,5,0
5,6,CIS-438,Database Administration,5,0
6,7,CIS-430,Business Systems Programming I,5,0
7,8,CIS-435,Business Systems Programming II,5,0


In [7]:
course_data_df.rename(columns={
    'CourseCode':'Course_Code__c',
    'CourseName':'Course_Name__c',
    'CreditHours':'Credit_Hours__c',
    'BootCampCourse':'Boot_Camp_Course__c',    
}, inplace=True)
course_data_df.head()

,ID_Course,Course_Code__c,Course_Name__c,Credit_Hours__c,Boot_Camp_Course__c
0,1,BC-DATAVIZ,Data Visualization and Analytics,12,1
1,2,BC-WEBDEV,Full Stack Web Development,12,1
2,3,BC-UIUX,User Interface/User Experience,12,1
3,4,CIS-349,Introduction to Databases,5,0
4,5,CIS-405,Database Programming,5,0


In [8]:
course_data_df = course_data_df[['Course_Code__c', 'Course_Name__c', 'Credit_Hours__c', 'Boot_Camp_Course__c']]
course_data_df.head()

,Course_Code__c,Course_Name__c,Credit_Hours__c,Boot_Camp_Course__c
0,BC-DATAVIZ,Data Visualization and Analytics,12,1
1,BC-WEBDEV,Full Stack Web Development,12,1
2,BC-UIUX,User Interface/User Experience,12,1
3,CIS-349,Introduction to Databases,5,0
4,CIS-405,Database Programming,5,0


In [9]:
course_data_records = course_data_df.to_dict('records')
course_data_records

[{'Course_Code__c': 'BC-DATAVIZ',
  'Course_Name__c': 'Data Visualization and Analytics',
  'Credit_Hours__c': 12,
  'Boot_Camp_Course__c': 1},
 {'Course_Code__c': 'BC-WEBDEV',
  'Course_Name__c': 'Full Stack Web Development',
  'Credit_Hours__c': 12,
  'Boot_Camp_Course__c': 1},
 {'Course_Code__c': 'BC-UIUX',
  'Course_Name__c': 'User Interface/User Experience',
  'Credit_Hours__c': 12,
  'Boot_Camp_Course__c': 1},
 {'Course_Code__c': 'CIS-349',
  'Course_Name__c': 'Introduction to Databases',
  'Credit_Hours__c': 5,
  'Boot_Camp_Course__c': 0},
 {'Course_Code__c': 'CIS-405',
  'Course_Name__c': 'Database Programming',
  'Credit_Hours__c': 5,
  'Boot_Camp_Course__c': 0},
 {'Course_Code__c': 'CIS-438',
  'Course_Name__c': 'Database Administration',
  'Credit_Hours__c': 5,
  'Boot_Camp_Course__c': 0},
 {'Course_Code__c': 'CIS-430',
  'Course_Name__c': 'Business Systems Programming I',
  'Credit_Hours__c': 5,
  'Boot_Camp_Course__c': 0},
 {'Course_Code__c': 'CIS-435',
  'Course_Name__c':

In [10]:
for rec in course_data_records:

    record = {
        'Course_Code__c': rec['Course_Code__c'],
        'Course_Name__c': rec['Course_Name__c'],
        'Credit_Hours__c': rec['Credit_Hours__c'],
        'Boot_Camp_Course__c': rec['Boot_Camp_Course__c'],
    }
    
    try:
        sf.Course__c.create(record)
    except Exception as e:
        print(e)

Malformed request https://dartanionnet2-dev-ed.my.salesforce.com/services/data/v42.0/sobjects/Course__c/. Response content: [{'message': 'duplicate value found: Course_Code__c duplicates value on record with id: a034x0000034bds', 'errorCode': 'DUPLICATE_VALUE', 'fields': []}]
Malformed request https://dartanionnet2-dev-ed.my.salesforce.com/services/data/v42.0/sobjects/Course__c/. Response content: [{'message': 'duplicate value found: Course_Code__c duplicates value on record with id: a034x0000034bdx', 'errorCode': 'DUPLICATE_VALUE', 'fields': []}]
Malformed request https://dartanionnet2-dev-ed.my.salesforce.com/services/data/v42.0/sobjects/Course__c/. Response content: [{'message': 'duplicate value found: Course_Code__c duplicates value on record with id: a034x0000034be2', 'errorCode': 'DUPLICATE_VALUE', 'fields': []}]
Malformed request https://dartanionnet2-dev-ed.my.salesforce.com/services/data/v42.0/sobjects/Course__c/. Response content: [{'message': 'duplicate value found: Course_C

In [11]:
# Bulk 
# sf.bulk.Course__c.insert(course_data_records)

## Create Course Lookup Table
You will use this later to crosswalk the course code with the primary key from the `Course` table

It is important to note that we will be querying **Salesforce** to retrieve the record IDs 

In [12]:
course_lookup_list = []

# The `Name` column in the primary key in Salesforce objects
# The Salesforce query language is called SOQL 
data = sf.query_all_iter("SELECT Name, Course_Code__c FROM Course__c")
for row in data:
    rec = {
        'ID_Course__c': row['Name'], # this is a critical line of code
        'Course_Code__c': row['Course_Code__c']
    }
    course_lookup_list.append(rec)
    
course_lookup_list

[{'ID_Course__c': 'a034x0000034beC', 'Course_Code__c': 'CIS-405'},
 {'ID_Course__c': 'a034x0000034be2', 'Course_Code__c': 'BC-UIUX'},
 {'ID_Course__c': 'a034x0000034bds', 'Course_Code__c': 'BC-DATAVIZ'},
 {'ID_Course__c': 'a034x0000034beM', 'Course_Code__c': 'CIS-430'},
 {'ID_Course__c': 'a034x0000034be7', 'Course_Code__c': 'CIS-349'},
 {'ID_Course__c': 'a034x0000034beH', 'Course_Code__c': 'CIS-438'},
 {'ID_Course__c': 'a034x0000034beR', 'Course_Code__c': 'CIS-435'},
 {'ID_Course__c': 'a034x0000034bdx', 'Course_Code__c': 'BC-WEBDEV'}]

In [13]:
course_lookup_df = pd.DataFrame(course_lookup_list)
course_lookup_df

,ID_Course__c,Course_Code__c
0,a034x0000034beC,CIS-405
1,a034x0000034be2,BC-UIUX
2,a034x0000034bds,BC-DATAVIZ
3,a034x0000034beM,CIS-430
4,a034x0000034be7,CIS-349
5,a034x0000034beH,CIS-438
6,a034x0000034beR,CIS-435
7,a034x0000034bdx,BC-WEBDEV


In [14]:
# Query the `Class` table from MySQL
query = '''
    SELECT 
        co.CourseCode,
        cl.*
    FROM 
        class cl
        INNER JOIN course co
        ON cl.ID_Course = co.ID_Course

'''

class_data_df = pd.read_sql(query, conn)
class_data_df.head()

,CourseCode,ID_Class,ID_Course,Section,StartDate,EndDate
0,BC-DATAVIZ,1,1,GWU-ARL-DATA-PT-09-0,2020-09-16,2020-03-14
1,BC-DATAVIZ,2,1,GWDC201805DATA3,2018-05-15,2018-11-08
2,BC-WEBDEV,3,2,GWARL201905WEB3,2019-05-14,2019-11-07
3,BC-UIUX,4,3,GWARL201905UIUX3,2019-05-14,2019-11-07


In [15]:
class_data_df.rename(columns={
    'CourseCode':'Course_Code__c',
    'Section':'Section__c',
    'StartDate':'Start_Date__c',
    'EndDate':'End_Date__c',
    'CourseCode':'Course_Code__c'
}, inplace=True)


class_data_df = class_data_df[['Course_Code__c', 'Section__c', 'Start_Date__c', 'End_Date__c']]
class_data_df

,Course_Code__c,Section__c,Start_Date__c,End_Date__c
0,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,2020-09-16,2020-03-14
1,BC-DATAVIZ,GWDC201805DATA3,2018-05-15,2018-11-08
2,BC-WEBDEV,GWARL201905WEB3,2019-05-14,2019-11-07
3,BC-UIUX,GWARL201905UIUX3,2019-05-14,2019-11-07


## Join the Class DataFrame with the Course lookup table
This join is necessary to successfully lookup the foreign key for the Course table 

In [16]:
class_data_df = pd.merge(class_data_df, course_lookup_df, how='left')
#class_data_df.drop(columns = ['ID_Class','ID_Course','CourseName','CreditHours','BootCampCourse','Course_Code__c'], inplace=True)

class_data_df.head()

,Course_Code__c,Section__c,Start_Date__c,End_Date__c,ID_Course__c
0,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,2020-09-16,2020-03-14,a034x0000034bds
1,BC-DATAVIZ,GWDC201805DATA3,2018-05-15,2018-11-08,a034x0000034bds
2,BC-WEBDEV,GWARL201905WEB3,2019-05-14,2019-11-07,a034x0000034bdx
3,BC-UIUX,GWARL201905UIUX3,2019-05-14,2019-11-07,a034x0000034be2


In [17]:
#class_data_df
class_data_df['Start_Date__c'] = class_data_df['Start_Date__c'].astype(str)
class_data_df['End_Date__c'] = class_data_df['End_Date__c'].astype(str)

class_data_df.head()

,Course_Code__c,Section__c,Start_Date__c,End_Date__c,ID_Course__c
0,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,2020-09-16,2020-03-14,a034x0000034bds
1,BC-DATAVIZ,GWDC201805DATA3,2018-05-15,2018-11-08,a034x0000034bds
2,BC-WEBDEV,GWARL201905WEB3,2019-05-14,2019-11-07,a034x0000034bdx
3,BC-UIUX,GWARL201905UIUX3,2019-05-14,2019-11-07,a034x0000034be2


In [18]:
class_data_records = class_data_df.to_dict(orient='records')
class_data_records

[{'Course_Code__c': 'BC-DATAVIZ',
  'Section__c': 'GWU-ARL-DATA-PT-09-0',
  'Start_Date__c': '2020-09-16',
  'End_Date__c': '2020-03-14',
  'ID_Course__c': 'a034x0000034bds'},
 {'Course_Code__c': 'BC-DATAVIZ',
  'Section__c': 'GWDC201805DATA3',
  'Start_Date__c': '2018-05-15',
  'End_Date__c': '2018-11-08',
  'ID_Course__c': 'a034x0000034bds'},
 {'Course_Code__c': 'BC-WEBDEV',
  'Section__c': 'GWARL201905WEB3',
  'Start_Date__c': '2019-05-14',
  'End_Date__c': '2019-11-07',
  'ID_Course__c': 'a034x0000034bdx'},
 {'Course_Code__c': 'BC-UIUX',
  'Section__c': 'GWARL201905UIUX3',
  'Start_Date__c': '2019-05-14',
  'End_Date__c': '2019-11-07',
  'ID_Course__c': 'a034x0000034be2'}]

## Insert `Class` Records into Salesforce

In [19]:
for rec in class_data_records:
 
    record = {
        'ID_Course__c': rec['ID_Course__c'],
        'Section__c': rec['Section__c'],
        'Start_Date__c': rec['Start_Date__c'],
        'End_Date__c': rec['End_Date__c'],
    }
    
    try:
        sf.Class__C.create(record)
    except Exception as e:
        print(e)

# Query `Student` Records from MySQL

In [20]:
query = '''
    SELECT 
        StudentID AS Student_ID__c,
        LastName AS Last_Name__c,
        FirstName AS First_Name__c,
        MiddleName AS Middle_Name__c,
        BirthDate AS Birth_Date__c,
        Gender AS Gender__c
    FROM
        student
'''

student_df = pd.read_sql(query, conn)

print(student_df.head())

student_dict = student_df.to_dict(orient='records')

student_dict

  Student_ID__c Last_Name__c First_Name__c Middle_Name__c Birth_Date__c  \
0      25004961     Dartling       Heather          Alice          None   
1      25003514     Dartling          Lana        Cecille          None   
2      25005833     Dartling       Jessica        Dorothy          None   
3      25002589     Dartling      Kimberly      Genevieve          None   
4      25007185     Dartling     Katherine       Lynnette          None   

  Gender__c  
0         F  
1         F  
2         F  
3         F  
4         F  


[{'Student_ID__c': '25004961',
  'Last_Name__c': 'Dartling',
  'First_Name__c': 'Heather',
  'Middle_Name__c': 'Alice',
  'Birth_Date__c': None,
  'Gender__c': 'F'},
 {'Student_ID__c': '25003514',
  'Last_Name__c': 'Dartling',
  'First_Name__c': 'Lana',
  'Middle_Name__c': 'Cecille',
  'Birth_Date__c': None,
  'Gender__c': 'F'},
 {'Student_ID__c': '25005833',
  'Last_Name__c': 'Dartling',
  'First_Name__c': 'Jessica',
  'Middle_Name__c': 'Dorothy',
  'Birth_Date__c': None,
  'Gender__c': 'F'},
 {'Student_ID__c': '25002589',
  'Last_Name__c': 'Dartling',
  'First_Name__c': 'Kimberly',
  'Middle_Name__c': 'Genevieve',
  'Birth_Date__c': None,
  'Gender__c': 'F'},
 {'Student_ID__c': '25007185',
  'Last_Name__c': 'Dartling',
  'First_Name__c': 'Katherine',
  'Middle_Name__c': 'Lynnette',
  'Birth_Date__c': None,
  'Gender__c': 'F'},
 {'Student_ID__c': '25006014',
  'Last_Name__c': 'Dartling',
  'First_Name__c': 'Precious',
  'Middle_Name__c': 'Mariette',
  'Birth_Date__c': None,
  'Gender_

In [21]:
try:
    for rec in student_dict:
        sf.Student__c.create(rec)
except Exception as e:
    print(e)

Malformed request https://dartanionnet2-dev-ed.my.salesforce.com/services/data/v42.0/sobjects/Student__c/. Response content: [{'message': 'duplicate value found: Student_ID__c duplicates value on record with id: a054x000000gsEl', 'errorCode': 'DUPLICATE_VALUE', 'fields': []}]


# Class Participant Example 

In [22]:
sf_course_records = []

sf_course_data = sf.query('SELECT Name, Course_Code__c FROM Course__c')

for row in sf_course_data['records']:
    
    rec = {
        'ID_Course__c': row['Name'],
        'Course_Code__c': row['Course_Code__c']
    }
    
    sf_course_records.append(rec)
    
sf_course_df = pd.DataFrame(sf_course_records)
sf_course_df.head()

,ID_Course__c,Course_Code__c
0,a034x0000034beC,CIS-405
1,a034x0000034be2,BC-UIUX
2,a034x0000034bds,BC-DATAVIZ
3,a034x0000034beM,CIS-430
4,a034x0000034be7,CIS-349


In [38]:
sf_class_records = []

sf_class_data = sf.query('SELECT ID_Course__c, Name, Section__c FROM Class__c')

for row in sf_class_data['records']:
    
    rec = {
        'ID_Course__c': row['ID_Course__c'],
        'ID_Class__c': row['Name'],
        'Section__c': row['Section__c']
    }
    
    sf_class_records.append(rec)
    
# temporary workaround to strip last three characters from ID_Course__c
#for rec in sf_class_records:
#    rec['ID_Course__c'] = rec['ID_Course__c'][:-3]
    
sf_class_df = pd.DataFrame(sf_class_records)

# temporary workaround to strip last three characters from ID_Course__c 
sf_class_df['ID_Course__c'] = sf_class_df['ID_Course__c'].apply(lambda col: col[:-3])

sf_class_df.head()

,ID_Course__c,ID_Class__c,Section__c
0,a034x0000034bds,a044x000001xd5v,GWDC201805DATA3
1,a034x0000034bdx,a044x000001xcqe,GWARL201905WEB3
2,a034x0000034be2,a044x000001xcqj,GWARL201905UIUX3
3,a034x0000034bds,a044x000001xd5q,GWU-ARL-DATA-PT-09-0
4,a034x0000034bds,a044x000001xcqU,GWU-ARL-DATA-PT-09-0


In [24]:
sf_class_xwalk = pd.merge(sf_class_df, sf_course_df)
sf_class_xwalk

,ID_Course__c,ID_Class__c,Section__c,Course_Code__c
0,a034x0000034bds,a044x000001xd5v,GWDC201805DATA3,BC-DATAVIZ
1,a034x0000034bds,a044x000001xd5q,GWU-ARL-DATA-PT-09-0,BC-DATAVIZ
2,a034x0000034bds,a044x000001xcqU,GWU-ARL-DATA-PT-09-0,BC-DATAVIZ
3,a034x0000034bds,a044x000001xcqZ,GWDC201805DATA3,BC-DATAVIZ
4,a034x0000034bdx,a044x000001xcqe,GWARL201905WEB3,BC-WEBDEV
5,a034x0000034bdx,a044x000001xd60,GWARL201905WEB3,BC-WEBDEV
6,a034x0000034be2,a044x000001xcqj,GWARL201905UIUX3,BC-UIUX
7,a034x0000034be2,a044x000001xd65,GWARL201905UIUX3,BC-UIUX


In [25]:
sf_student_records = []

sf_student_data = sf.query('SELECT Name, Student_ID__c FROM Student__c')

for row in sf_student_data['records']:
    
    rec = {
        'ID_Student__c': row['Name'],
        'Student_ID__c': row['Student_ID__c']
    }
    
    sf_student_records.append(rec)
    
sf_student_xwalk = pd.DataFrame(sf_student_records)
sf_student_xwalk.head()

,ID_Student__c,Student_ID__c
0,a054x000000gsFA,25006014
1,a054x000000gsFe,25002311
2,a054x000000gsEq,25003514
3,a054x000000gsGw,25007333
4,a054x000000gsFF,25007528


In [26]:
query = '''

    SELECT
        s.StudentID AS Student_ID__c,
        co.CourseCode AS Course_Code__c,
        cl.Section AS Section__c,
        cp.StartDate AS Start_Date__c,
        cp.EndDate AS End_Date__c
    FROM 
        classparticipant cp
        INNER JOIN class cl
        ON cp.ID_Class = cl.ID_Class
        INNER JOIN course co
        ON cl.ID_Course = co.ID_Course
        INNER JOIN student s
        ON cp.ID_Student = s.ID_Student

'''

classparticiant_df = pd.read_sql(query, conn)
classparticiant_df.head()

,Student_ID__c,Course_Code__c,Section__c,Start_Date__c,End_Date__c
0,25004961,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,2020-09-16,None
1,25003514,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,2020-09-16,None
2,25005833,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,2020-09-16,None
3,25007334,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,2020-09-16,None
4,25002589,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,2020-09-16,None


In [27]:
#sf_class_xwalk
#sf_student_xwalk

class_participant_load = pd.merge(classparticiant_df, sf_class_xwalk, on=['Course_Code__c', 'Section__c'])

class_participant_load = pd.merge(class_participant_load, sf_student_xwalk)

class_participant_load = class_participant_load[['ID_Student__c','ID_Class__c','Start_Date__c','End_Date__c']]

class_participant_load['Start_Date__c'] = class_participant_load['Start_Date__c'].astype(str)

class_participant_load

class_participant_records = class_participant_load.to_dict(orient='records')
class_participant_records

[{'ID_Student__c': 'a054x000000gsEl',
  'ID_Class__c': 'a044x000001xd5q',
  'Start_Date__c': '2020-09-16',
  'End_Date__c': None},
 {'ID_Student__c': 'a054x000000gsEl',
  'ID_Class__c': 'a044x000001xcqU',
  'Start_Date__c': '2020-09-16',
  'End_Date__c': None},
 {'ID_Student__c': 'a054x000000gsEq',
  'ID_Class__c': 'a044x000001xd5q',
  'Start_Date__c': '2020-09-16',
  'End_Date__c': None},
 {'ID_Student__c': 'a054x000000gsEq',
  'ID_Class__c': 'a044x000001xcqU',
  'Start_Date__c': '2020-09-16',
  'End_Date__c': None},
 {'ID_Student__c': 'a054x000000gsEv',
  'ID_Class__c': 'a044x000001xd5q',
  'Start_Date__c': '2020-09-16',
  'End_Date__c': None},
 {'ID_Student__c': 'a054x000000gsEv',
  'ID_Class__c': 'a044x000001xcqU',
  'Start_Date__c': '2020-09-16',
  'End_Date__c': None},
 {'ID_Student__c': 'a054x000000gsH1',
  'ID_Class__c': 'a044x000001xd5q',
  'Start_Date__c': '2020-09-16',
  'End_Date__c': None},
 {'ID_Student__c': 'a054x000000gsH1',
  'ID_Class__c': 'a044x000001xcqU',
  'Start_

In [28]:
for row in class_participant_records:
    try:
        sf.Class_Participant__c.create(row)
    except Exception as e:
        print(e)

## Example of Deleting Records

Select the IDs of the records first and then process the results.

Ultimately, you want a list of IDs in the end.


In [29]:
class_records = sf.query("SELECT Name FROM Student__c")
recs_to_delete = [{'Id': r['Name']} for r in class_records['records']]
recs_to_delete

[{'Id': 'a054x000000gsFA'},
 {'Id': 'a054x000000gsFe'},
 {'Id': 'a054x000000gsEq'},
 {'Id': 'a054x000000gsGw'},
 {'Id': 'a054x000000gsFF'},
 {'Id': 'a054x000000gsGh'},
 {'Id': 'a054x000000gsG8'},
 {'Id': 'a054x000000gsF0'},
 {'Id': 'a054x000000gsGN'},
 {'Id': 'a054x000000gsGI'},
 {'Id': 'a054x000000gsEl'},
 {'Id': 'a054x000000gsFt'},
 {'Id': 'a054x000000gsG3'},
 {'Id': 'a054x000000gsGX'},
 {'Id': 'a054x000000gsEX'},
 {'Id': 'a054x000000gsGD'},
 {'Id': 'a054x000000gsF5'},
 {'Id': 'a054x000000gsGS'},
 {'Id': 'a054x000000gsH1'},
 {'Id': 'a054x000000gsFj'},
 {'Id': 'a054x000000gsEv'},
 {'Id': 'a054x000000gsFU'},
 {'Id': 'a054x000000gsFo'},
 {'Id': 'a054x000000gsFK'},
 {'Id': 'a054x000000gsGm'},
 {'Id': 'a054x000000gsGr'},
 {'Id': 'a054x000000gsFy'},
 {'Id': 'a054x000000gsFZ'},
 {'Id': 'a054x000000gsGc'},
 {'Id': 'a054x000000gsFP'}]

In [30]:
#sf.bulk.Course__c.delete(recs_to_delete)

In [31]:
for rec in recs_to_delete:
    try:
        sf.Student__c.delete(rec['Id'])
    except Exception as e:
        print(e)

Malformed request https://dartanionnet2-dev-ed.my.salesforce.com/services/data/v42.0/sobjects/Student__c/a054x000000gsFA. Response content: [{'message': 'Your attempt to delete a054x000000gsFA could not be completed because it is associated with the following class participants.: a074x000001iS7H, a074x000001iSQc, a074x000001iSQh\n', 'errorCode': 'DELETE_FAILED', 'fields': []}]
Malformed request https://dartanionnet2-dev-ed.my.salesforce.com/services/data/v42.0/sobjects/Student__c/a054x000000gsEq. Response content: [{'message': 'Your attempt to delete a054x000000gsEq could not be completed because it is associated with the following class participants.: a074x000001iS6s, a074x000001iSPo, a074x000001iSPt\n', 'errorCode': 'DELETE_FAILED', 'fields': []}]
Malformed request https://dartanionnet2-dev-ed.my.salesforce.com/services/data/v42.0/sobjects/Student__c/a054x000000gsGw. Response content: [{'message': 'Your attempt to delete a054x000000gsGw could not be completed because it is associated

Malformed request https://dartanionnet2-dev-ed.my.salesforce.com/services/data/v42.0/sobjects/Student__c/a054x000000gsGr. Response content: [{'message': 'Your attempt to delete a054x000000gsGr could not be completed because it is associated with the following class participants.: a074x000001iS8o, a074x000001iSTg, a074x000001iSTl\n', 'errorCode': 'DELETE_FAILED', 'fields': []}]
Malformed request https://dartanionnet2-dev-ed.my.salesforce.com/services/data/v42.0/sobjects/Student__c/a054x000000gsFy. Response content: [{'message': 'Your attempt to delete a054x000000gsFy could not be completed because it is associated with the following class participants.: a074x000001iS80, a074x000001iSS4, a074x000001iSS9\n', 'errorCode': 'DELETE_FAILED', 'fields': []}]
Malformed request https://dartanionnet2-dev-ed.my.salesforce.com/services/data/v42.0/sobjects/Student__c/a054x000000gsFZ. Response content: [{'message': 'Your attempt to delete a054x000000gsFZ could not be completed because it is associated